In [1]:

import torch
import torchvision
import math
import pandas as pd
import numpy as np
import gc

from tqdm import tqdm_notebook
from torch.utils.tensorboard import SummaryWriter


from modules.GWD_data import WheatDataset
from modules.metric import calculate_image_precision
from modules.utils import collate_fn, plot_boxes, get_model_name, format_prediction_string, remove_blanks, is_contain_blanks
from config import config


In [2]:
conf = config()
torch.random.manual_seed(5)
writer = SummaryWriter("runs/Edet_4_Test_run") #


WD_Train = WheatDataset(conf)
WD_Valid = WheatDataset(conf, train=False)
WD_Train_Loader = torch.utils.data.DataLoader(WD_Train, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)
WD_Valid_Loader = torch.utils.data.DataLoader(WD_Valid, batch_size=conf.BATH_SIZE, shuffle=True, collate_fn=collate_fn)

In [3]:
def get_model(num_classes=2):
    print('New Model ')
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=True, pretrained_backbone=True)
    model.roi_heads.box_predictor.cls_score = torch.nn.Linear(model.roi_heads.box_predictor.cls_score.in_features, num_classes, bias=True)
    model.roi_heads.box_predictor.bbox_pred = torch.nn.Linear(model.roi_heads.box_predictor.bbox_pred.in_features, num_classes*4, bias=True)
    return model

In [5]:
#model_path=None
#GWD_Model = torch.load(model_path) if model_path else get_model()
#GWD_Model.load_state_dict(torch.load('model/model_rc/fasterrcnn_resnet50_fpn_best.pth'))
#GWD_Model.to(conf.DEVICE)
#EPOCH = int(model_path.split('/')[-1].split('_')[2])+1 if model_path else 0

#params = [p for p in GWD_Model.parameters() if p.requires_grad]
#optimizer = torch.optim.SGD(params, lr=0.0002,momentum=0.9, weight_decay=0.0005)

New Model 


In [4]:
model_path = 'model/model_rc/GWD_EPOCH_19_SCORE_0.0000_LOSS_201.4089.pt'
GWD_Model = torch.load(model_path) if model_path else get_model()
GWD_Model.to(conf.DEVICE)
EPOCH = int(model_path.split('/')[-1].split('_')[2])+1 if model_path else 0

params = [p for p in GWD_Model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.001,momentum=0.9, weight_decay=0.0005)


In [5]:
_iter=len(WD_Train_Loader)*(EPOCH)

for i in tqdm_notebook(range(EPOCH, 30, 1)):
    
    _epoch_loss=0
    _ = GWD_Model.train()
    for images, targets in tqdm_notebook(WD_Train_Loader):
        
        #if not is_contain_blanks(targets):
        try:
            images = [torch.tensor(image, dtype = torch.float32).to(conf.DEVICE) for image in images]
            targets = [{k: torch.tensor(v).to(conf.DEVICE) for k, v in target.items()} for target in targets]

            loss_dictionary = GWD_Model(images,targets)
            summed_loss = sum(loss for loss in loss_dictionary.values())
            summed_loss_value = summed_loss.item()


            if math.isfinite(summed_loss_value):
                optimizer.zero_grad()
                summed_loss.backward()
                optimizer.step()
                _epoch_loss+=summed_loss_value

            else:
                print('Loss is undefined:',summed_loss_value,'   skipping BackProp for step no:',_iter)
                print(loss_dictionary)


            writer.add_scalar('Running Loss/Summed', summed_loss_value, _iter)
            writer.add_scalar('Running Loss/Classifier', loss_dictionary['loss_classifier'].item(), _iter)
            writer.add_scalar('Running Loss/Box_Regress', loss_dictionary['loss_box_reg'].item(), _iter)
            writer.add_scalar('Running Loss/Objectness', loss_dictionary['loss_objectness'].item(), _iter)
            writer.add_scalar('Running Loss/RPN_Box_regress', loss_dictionary['loss_rpn_box_reg'].item(), _iter)
            
        except:
            print('Some weird shape error due to blank images, but handeled')
        
        _iter+=1
    scheduler.step()  
    torch.save(GWD_Model, 'model/model_rc/'+get_model_name(i, 0, _epoch_loss))
    torch.cuda.empty_cache()
    gc.collect()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
..\torch\csrc\utils\python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



...

Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled



Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled
Some weird shape error due to blank images, but handeled




# EDet-4

In [1]:
import torch
from torchvision import transforms
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import traceback
import gc
import sys
sys.path.insert(1, 'models/EfficientDet')

In [2]:
from edet import edet#, freeze_backbone
from efficientdet.loss import FocalLoss
from modules.edet_utils import feeze_backbone
from modules.edet_utils import WheatDataset_edet, Resizer, Normalizer, collater
from config import config
conf = config()

In [3]:
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
obj_list = ['wheat']
compound_coef = 4
model = edet(compound_coef, obj_list)
weight_dir = 'D:\Projects\Kaggle\Global-Wheat-Detction-Kaggle-2020\saved_weights\Edet\d'+str(compound_coef)

In [4]:
saved_weight_path = weight_dir+'\last.pth'
try:
    ret = model.load_state_dict(torch.load(saved_weight_path) , strict=False)
except RuntimeError as e:
    print(f'[Warning] Ignoring {e}')
    print('\n\nThis error caused because last layer size don\'t match')

In [5]:
optimizer = torch.optim.AdamW(model.parameters(), 0.0005)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)

In [7]:
mean= [0.485, 0.456, 0.406]
std= [0.229, 0.224, 0.225]

Data_Transform = transforms.Compose(
    [Normalizer(mean=mean, std=std),
     Resizer(input_sizes[compound_coef])])

WD_Train = WheatDataset_edet(conf, transform = Data_Transform)
WD_Valid = WheatDataset_edet(conf, train=False)
WD_Train_Loader = DataLoader(WD_Train,
                             batch_size=conf.BATCH_SIZE,
                             shuffle=True,
                             collate_fn=collater)
WD_Valid_Loader = DataLoader(WD_Valid,
                             batch_size=conf.BATCH_SIZE,
                             shuffle=True,
                             collate_fn=collater)

In [8]:
WD_Train[0]

{'img': tensor([[[-1.6727, -1.6506, -1.4036],
          [-1.6042, -1.5805, -1.3339],
          [-1.5014, -1.4755, -1.2293],
          ...,
          [-1.0390, -0.3375, -0.6367],
          [-1.0219, -0.3375, -0.6541],
          [-1.0390, -0.3375, -0.6715]],
 
         [[-1.7069, -1.6856, -1.4384],
          [-1.6555, -1.6331, -1.3861],
          [-1.5528, -1.5280, -1.2816],
          ...,
          [-1.0562, -0.4251, -0.6367],
          [-1.1075, -0.4076, -0.7064],
          [-1.1075, -0.4076, -0.7064]],
 
         [[-1.7583, -1.7206, -1.4907],
          [-1.7069, -1.6856, -1.4384],
          [-1.6213, -1.5980, -1.3513],
          ...,
          [-1.0733, -0.4426, -0.6018],
          [-1.0733, -0.4426, -0.6018],
          [-1.0733, -0.4426, -0.6018]],
 
         ...,
 
         [[-1.1589, -1.0903, -0.8807],
          [-1.1589, -1.0903, -0.8807],
          [-1.1589, -1.0903, -0.8807],
          ...,
          [ 1.6153,  1.8333,  0.7402],
          [ 1.6667,  1.8859,  0.8274],
          [

In [9]:
class ModelWithLoss(nn.Module):
    def __init__(self, model, debug=False):
        super().__init__()
        self.criterion = FocalLoss()
        self.model = model
        self.debug = debug

    def forward(self, imgs, annotations, obj_list=None):
        _, regression, classification, anchors = self.model(imgs)
        if self.debug:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations,
                                                imgs=imgs, obj_list=obj_list)
        else:
            cls_loss, reg_loss = self.criterion(classification, regression, anchors, annotations)
        return cls_loss, reg_loss
model = ModelWithLoss(model)

In [11]:
writer = SummaryWriter("runs/Edet_4_Test_run")
model.train()
model.to(conf.DEVICE)

last_epoch = 0
# TODO: Extract last epoch from saved model name
current_iter = max(0, last_epoch*len(WD_Train_Loader))

for epoch in range(last_epoch, 2, 1):

    epoch_loss = []
    progress_bar = tqdm(WD_Train_Loader)
        
    for _, data in enumerate(progress_bar):
        
        
        

        # TODO: targets to required format
        try:
            
            images = data['img']
            targets = data['annot']

            images = images.to(conf.DEVICE)
            targets = targets.to(conf.DEVICE)
            
            optimizer.zero_grad()

            cls_loss, reg_loss = model(images, targets, obj_list=obj_list)
            cls_loss = cls_loss.mean()
            reg_loss = reg_loss.mean()

            loss = cls_loss + reg_loss
            if loss == 0 or not torch.isfinite(loss):
                print('loss is 0 or infinite')
                continue

            loss.backward()
            optimizer.step()
            epoch_loss.append(float(loss))

            current_iter+=1
            writer.add_scalar('Running Loss/Summed', loss, current_iter)
            writer.add_scalar('Running Loss/Classifier', cls_loss, current_iter)
            writer.add_scalar('Running Loss/Box_Regress', reg_loss, current_iter)
            writer.add_scalar('Running LR', optimizer.param_groups[0]['lr'], current_iter)

            progress_bar.set_description(
                'Epoch: {}, Images: {}, Cls_loss: {:.5f}, Reg_loss: {:.5f}, Total_loss: {:.5f}'
                .format(epoch, _ * conf.BATCH_SIZE, cls_loss.item(),reg_loss.item(), loss.item()))

            
            
        except Exception as e:
            print('[Error]', traceback.format_exc())
            print(e)
            continue
            
    save_model_path = 'GWD_EDET{}_EPOCH_{}_SCORE_{:.5f}_LOSS_{:.5f}.pth'.format(compound_coef, epoch, 0, np.mean(epoch_loss))
    torch.save(model.state_dict(), weight_dir+'/'+save_model_path)
    torch.save(model.state_dict(), weight_dir+'/last.pth')
    print('model_saved')
    scheduler.step(np.mean(epoch_loss))
    torch.cuda.empty_cache()
    gc.collect()
    

    

[Error] Traceback (most recent call last):
  File "<ipython-input-11-98f2bf198274>", line 30, in <module>
    cls_loss, reg_loss = model(images, targets, obj_list=obj_list)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-9-b16f1436717e>", line 9, in forward
    _, regression, classification, anchors = self.model(imgs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "models/EfficientDet\backbone.py", line 65, in forward
    _, p3, p4, p5 = self.backbone_net(inputs)
  File "C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\module.py", line 550, in __call__
    result = self.forward(*input, **kwargs)
  File "models/EfficientDet\efficientdet\model.py", line 412, in forward
    x = block(x, drop_connect_rate=drop_connect_rate)
  File "C:\ProgramData\Anaconda3


CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 12.00 GiB total capacity; 9.30 GiB already allocated; 18.83 MiB free; 9.46 GiB reserved in total by PyTorch)


KeyboardInterrupt: 

# DETR

In [3]:
'saved_weights/fasterrcnn/GWD_FRCNN_0_EPOCH_0_SCORE_0_LOSS_0.pth'.split('/')[-1].split('_')

['GWD', 'FRCNN', '0', 'EPOCH', '0', 'SCORE', '0', 'LOSS', '0.pth']

In [1]:
import sys
sys.path.insert(1, 'models/detr')

In [2]:
from models.backbone import build_backbone
from models.transformer import build_transformer

In [20]:
class args():
    #device = 'cuda'
    lr=1e-4
    lr_backbone=1e-5
    batch_size=2
    weight_decay=1e-4
    epochs=300
    lr_drop=200
    clip_max_norm=0.1
    frozen_weights', type=str, default=None
    backbone', default='resnet50
    dilation', action='store_true
    position_embedding', default='sine
    enc_layers', default=6
    dec_layers', default=6
    dim_feedforward', default=2048
    hidden_dim', default=256
    dropout', default=0.1
    nheads', default=8
    num_queries', default=100
    pre_norm', action='store_true'
    masks', action='store_true
    no_aux_loss', dest='aux_loss', action='store_false'
    set_cost_class', default=1
    set_cost_bbox', default=5
    set_cost_giou', default=2
    mask_loss_coef', default=1
    dice_loss_coef', default=1
    bbox_loss_coef', default=5
    giou_loss_coef', default=2
    eos_coef', default=0.1
    dataset_file', default='coco'
    
    coco_path', type=str)
    parser.add_argument('--coco_panoptic_path', type=str)
    parser.add_argument('--remove_difficult', action='store_true')
    
    output_dir', default=''
    device', default='cuda'
    seed', default=42
    resume', default=''
    start_epoch', default=0
    eval', action='store_true'
    num_workers', default=2
    world_size', default=1
    dist_url', default='env://'
        

In [21]:
args = args()
args.device

'cuda'

In [21]:
from models.position_encoding import PositionEmbeddingSine, PositionEmbeddingLearned
from models.backbone import Backbone, Joiner
from models.transformer import Transformer
from models.detr import DETR

In [20]:
N_steps = 256//2
position_embedding = PositionEmbeddingLearned(N_steps)
#position_embedding = PositionEmbeddingSine(N_steps, normalize=True)
backbone = Backbone('resnet101', True, False, True)  #(name, train_backbone, inter_layers(for seg.), dialation)
model = Joiner(backbone, position_embedding)
model.num_channels = backbone.num_channels

In [18]:
trans = Transformer(d_model=256,
                    dropout=0.1,
                    nhead=8,
                    dim_feedforward=2048,
                    num_encoder_layers=6,
                    num_decoder_layers=6,
                    normalize_before=True)

In [23]:
model_fn = DETR(model, trans, num_classes=1, num_queries=100, aux_loss=True)

In [24]:
model_fn

DETR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Drop

In [8]:
import argparse

In [9]:
parser = argparse.ArgumentParser('DETR training and evaluation script', parents=[get_args_parser()])

In [10]:
args = parser.parse_args('')

In [11]:
args

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path=None, dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_queries=100, num_workers=2, output_dir='', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)

In [26]:
from models.backbone import build_backbone
from models.transformer import build_transformer
from main import get_args_parser
from models.detr import build